# TP 2 Deep Learning
## Visualisation avec TensorBoard

Lors du TP précédent, nous avons vu comment charger des données, les utiliser via un modèle que nous définissons comme une sous-classe de nn.Module, entraîner ce modèle sur des données d'entraînement et le tester sur des données de test. Pour suivre sa progression, nous avons affiché quelques statistiques au fur et à mesure que le modèle s'entraîne pour avoir une idée de la progression de l'entraînement. Cependant, nous pouvons faire beaucoup mieux que cela avec PyTorch. PyTorch intègre TensorBoard, qui est un outil conçu pour visualiser les résultats des entraînements des réseaux de neurones. Dans ce TP, nous allons illustrer certaines de ses fonctionnalités, en utilisant un autre jeux de données [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) qui est aussi intégré dans PyTorch à l'aide de `torchvision.datasets`. Fashion-MNIST est un jeux de données d'images d'articles de Zalando, composé d'un ensemble d'entraîînement de 60 000 exemples et d'un ensemble de test de 10 000 exemples. Chaque exemple est une image en niveaux de gris 28x28 pixels, associée à une étiquette de 10 classes. 

Ce TP abordera les points suivants : 

1.   Mettre en oeuvre TendorBoard
2.   Ecrire sur TensorBoard
3.   Analyser l'architecture d'un modèl en utilisant TensorBoard
4.   Travailler avec TensorBoard d'une manière interactive
  - avec des manières différentes d'analyser les donées d'entrainement
  - suivi de la perfomrance d'un modèle durant son entraîînement
  - évaluation des performances du modèle après son entraement. 

Dans ce TP nous utiluseront le jeux de données [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist). Pour la partie modèle, nous avons besoin d'importer trois parties de modules :

1. Nous commençons par importer les modules qui servent à afficher les images et les graphiques.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

2. Puis les modules (`torchvision`) utilisés pour telecharger les jeux de données, télécharger des modèles de CNN, appliquer des transormations sur les images pour la vision par ordinateur (`torchvision.transforms`). 

In [ ]:
import torchvision
import torchvision.transforms as transforms

3. Finalement, les modules utilisés pour la conception et l'entraîînement du modèle de CNN utilisé pour la classification.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Maintenant que nous avons importé les modules necéssaires pour le jeux de données et le modèle de CNN, nous commençons par préparer un `transform` qui permet composer plusieurs transormations à l'aide de `Compose`. Les transformations necessaires sont la connvertion des images en `tensors` puis la normalisation de ces derniers.

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

### Préparation du dataset
Nous pouvons télécharger les donneés d'enraînement et les données de test du jeux de données. La transformation est donc appliquée et passée en paramètre à la méthode du dataset. Pour distinguer les données d'entraînement des données de test, il suffit de mettre ta variable `train` à  `True`.  

In [ ]:
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

Les données d'entraînement et de test sont ensuite chargées pour qu'elles soit directement [exploiatables par PyTorch](https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel#). 

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

Avant de proceder à la conception et l'implémentation du model de CNN, nous devons dabord définir les 10 classes de vêtements du dataset [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist):

| Label | Description |
|-------|-------------|
| 0     | T-shirt/top |
| 1     | Trouser     |
| 2     | Pullover    |
| 3     | Dress       |
| 4     | Coat        |
| 5     | Sandal      |
| 6     | Shirt       |
| 7     | Sneaker     |
| 8     | Bag         |
| 9     | Ankle boot  |

In [ ]:
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

### Definition du CNN de classification
Après avoir préparé les données d'éntrainement et de test, nous pouvons dès à présent définir notre CNN de classification de vêêtements :
- ***Nombres de caneaux*** : [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) est un dataset d'images RGB (i.e. 3 canaux) alors que [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) sont des images niveau de gris (i.e. 1 canal).
- ***Taille d'image*** : dans [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), les images sont de taille de 32x32 pixels, alors que dans  [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) les images sont de taille de 28x28 pixels.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

### Exercice
Après avoir défini le CNN, realisez :
1. Une fonction qui affiche un extrait du jeux de données. Il faut installer `matplotlib` pour ceci (dans un terminal) : `pip3 install matplotlib`
2. L'apprentissage du réseau :  définir l'algorithme d'optimisation `optimizer`, définir le critère d'optimisation `criterion`, entraîner le modèle, puis enregistrer le modèle entraîné dans le fichier `./MNIST_fashion_net.pth`. 
3. (à faire après le TP 3)Comparez les deux modèles (TP2 et TP3): en terme de précision, temps d'inférence sur CPU, et sur GPU.

### Mise en oeuvre de [TensorBoard](https://www.tensorflow.org/tensorboard/)
TensorBoard est  fournit les solutions de visualisation et les outils nécessaires aux tests de machine learning :
- Suivi et visualisation de métriques telles que la perte et la justesse
- Visualisation du graphe de modèle (opérations et couches)
- Affichage d'histogrammes de pondérations, de biais ou d'autres Tensors au fur et à mesure de leur évolution
- Projection de représentations vectorielles continues dans un espace à plus faible dimension
- Affichage d'images, de texte et de données audio
- Profilage de programmes TensorFlow
- ...

Nous allons maintenant appeler `tensorboard` de `torch.utils` puis définir `SummaryWriter`. 
La classe `SummaryWriter` est le point d'entrée pour enregister les données afin de les visualiser en utilisant `tensorboard`. 
Le répértoire par défaut qui contiendra les logs est le répértoire `runs`. Nous pouvons dévinir un sous repèrtoire propore à notre TP. 
Pour pouvoir visualiser les résultats sur `tensorboard`, il faut l'installer: `pip3 install tensorboard`.

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/TP3_fashion_mnist')

### Ecrire dans TensorBoard
Comme premier exemple, nous allons afficher quelques images dans TensorBoard en utilisant la méthode `torchvision.utils.make_grid` qui permet de créer une grille d'images. Cette grille sera affichée par `imshow` de `matplotlob`.




In [ ]:
# Extraire quelques images des données d'entraînement
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Créer la grille de ces images
img_grid = torchvision.utils.make_grid(images)

to_be_shown = img_grid.numpy().transpose(1,2,0)
# Afficher la grille avec matplotlib
plt.imshow(to_be_shown)

La grille d'image doit être écrite dans `tensorboard` pour pouvoir la manipuler.

In [ ]:
writer.add_image('fashion_mnist_images_example', img_grid)

Dans un terminal, lancer la commande suivante:

In [ ]:
tensorboard --logdir=runs

Le TP4 sera consacré à la manipulation de `tensorboard` et l'intérprétation des résultats. 